# **Neural Network for Text Classification**
*   Implementation of a classic neural network for text classification supporting **multi-precision** training.
    *   Implementation currently supports training in either double, single, or half precision.
    *   This implies that both the computations and parameter storage are done in the specified precision.
*   Implementation of a classic neural network for text classification supporting **mixed-precision** training.
    *   Implementation currently supports half precision computations with single precision parameter storage.
*   Implementations are based off of TensorFlow's very own example: [TensorFlow Text Classification Tutorial](https://www.tensorflow.org/tutorials/keras/text_classification)

In [1]:
import time
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras import layers, models

In [2]:
tfds.disable_progress_bar()
# Set a global random seed
tf.random.set_seed(12)

In [3]:
# Define number of training runs to compute the average training time over
NUM_TRAINING_RUNS = 3
# Values are specific to the sentiment analysis dataset
VOCAB_SIZE = 1000
EMBEDDING_DIM = 64
BUFFER_SIZE = 10000
BATCH_SIZE = 64

In [12]:
def build_and_train(train_ds, encoder, precision):
    if precision == 'double':
        dtype = tf.float64
    elif precision == 'single':
        dtype = tf.float32
    else: # half
        dtype = tf.float16

    model = models.Sequential([
        encoder,
        layers.Embedding(VOCAB_SIZE, EMBEDDING_DIM, dtype=dtype),
        layers.Dropout(0.2, dtype=dtype), # !!!
        layers.GlobalAveragePooling1D(dtype=dtype),
        layers.Dropout(0.2, dtype=dtype), # !!!
        layers.Dense(1, activation='sigmoid', dtype=dtype)
    ])
    model.compile(optimizer='adam',
                  loss=tf.keras.losses.BinaryCrossentropy(),
                  metrics=['accuracy'])

    start_time = time.time()
    model.fit(train_ds, epochs=5)
    end_time = time.time()
    training_time = end_time - start_time

    return model, training_time

In [10]:
def build_and_train_mixed(train_ds, encoder):
    tf.keras.mixed_precision.set_global_policy('mixed_float16')

    model = models.Sequential([
        encoder,
        layers.Embedding(VOCAB_SIZE, EMBEDDING_DIM),
        layers.Dropout(0.2),
        layers.GlobalAveragePooling1D(),
        layers.Dropout(0.2),
        layers.Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer='adam',
                  loss=tf.keras.losses.BinaryCrossentropy(),
                  metrics=['accuracy'])

    start_time = time.time()
    model.fit(train_ds, epochs=5)
    end_time = time.time()
    training_time = end_time - start_time

    tf.keras.mixed_precision.set_global_policy('float32')
    return model, training_time

In [7]:
# Load dataset and split into train and test sets
dataset, info = tfds.load('imdb_reviews', with_info=True, as_supervised=True)
train_dataset, test_dataset = dataset['train'], dataset['test']
train_dataset = train_dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
test_dataset = test_dataset.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


In [8]:
# Create the text encoder
encoder = tf.keras.layers.TextVectorization(max_tokens=VOCAB_SIZE)
encoder.adapt(train_dataset.map(lambda text, label: text))

In [13]:
# Test run to make sure that everything is working properly before starting actual measurements
_ = build_and_train(train_dataset, encoder, precision='single')
_ = build_and_train_mixed(train_dataset, encoder)

Epoch 1/5
391/391 [==============================] - 19s 42ms/step - loss: 0.6824 - accuracy: 0.5886
Epoch 2/5
391/391 [==============================] - 6s 15ms/step - loss: 0.6280 - accuracy: 0.7195
Epoch 3/5
391/391 [==============================] - 5s 13ms/step - loss: 0.5577 - accuracy: 0.7754
Epoch 4/5
391/391 [==============================] - 4s 9ms/step - loss: 0.4976 - accuracy: 0.8081
Epoch 5/5
391/391 [==============================] - 5s 12ms/step - loss: 0.4508 - accuracy: 0.8290
Epoch 1/5
391/391 [==============================] - 15s 31ms/step - loss: 0.6821 - accuracy: 0.6000
Epoch 2/5
391/391 [==============================] - 5s 12ms/step - loss: 0.6289 - accuracy: 0.7242
Epoch 3/5
391/391 [==============================] - 6s 14ms/step - loss: 0.5566 - accuracy: 0.7762
Epoch 4/5
391/391 [==============================] - 4s 11ms/step - loss: 0.4946 - accuracy: 0.8118
Epoch 5/5
391/391 [==============================] - 6s 15ms/step - loss: 0.4509 - accuracy: 0.8301

In [14]:
# Train with double precision
time_double = 0.0
for _ in range(NUM_TRAINING_RUNS):
    model_double, training_time = build_and_train(train_dataset, encoder, 'double')
    time_double += training_time
accuracy_double = model_double.evaluate(test_dataset, verbose=2)[1]

Epoch 1/5
391/391 [==============================] - 27s 59ms/step - loss: 0.6827 - accuracy: 0.5941
Epoch 2/5
391/391 [==============================] - 8s 20ms/step - loss: 0.6288 - accuracy: 0.7156
Epoch 3/5
391/391 [==============================] - 5s 13ms/step - loss: 0.5572 - accuracy: 0.7722
Epoch 4/5
391/391 [==============================] - 5s 13ms/step - loss: 0.4967 - accuracy: 0.8108
Epoch 5/5
391/391 [==============================] - 5s 13ms/step - loss: 0.4517 - accuracy: 0.8281
Epoch 1/5
391/391 [==============================] - 22s 53ms/step - loss: 0.6826 - accuracy: 0.5986
Epoch 2/5
391/391 [==============================] - 7s 18ms/step - loss: 0.6313 - accuracy: 0.7231
Epoch 3/5
391/391 [==============================] - 5s 13ms/step - loss: 0.5605 - accuracy: 0.7711
Epoch 4/5
391/391 [==============================] - 6s 15ms/step - loss: 0.4991 - accuracy: 0.8077
Epoch 5/5
391/391 [==============================] - 5s 13ms/step - loss: 0.4536 - accuracy: 0.827

In [15]:
# Train with single precision
time_single = 0.0
for _ in range(NUM_TRAINING_RUNS):
    model_single, training_time = build_and_train(train_dataset, encoder, 'single')
    time_single += training_time
accuracy_single = model_single.evaluate(test_dataset, verbose=2)[1]

Epoch 1/5
391/391 [==============================] - 13s 30ms/step - loss: 0.6827 - accuracy: 0.5978
Epoch 2/5
391/391 [==============================] - 5s 12ms/step - loss: 0.6294 - accuracy: 0.7156
Epoch 3/5
391/391 [==============================] - 5s 12ms/step - loss: 0.5562 - accuracy: 0.7765
Epoch 4/5
391/391 [==============================] - 4s 10ms/step - loss: 0.4981 - accuracy: 0.8079
Epoch 5/5
391/391 [==============================] - 4s 10ms/step - loss: 0.4508 - accuracy: 0.8268
Epoch 1/5
391/391 [==============================] - 12s 27ms/step - loss: 0.6824 - accuracy: 0.6043
Epoch 2/5
391/391 [==============================] - 6s 15ms/step - loss: 0.6287 - accuracy: 0.7087
Epoch 3/5
391/391 [==============================] - 4s 10ms/step - loss: 0.5557 - accuracy: 0.7784
Epoch 4/5
391/391 [==============================] - 4s 10ms/step - loss: 0.4939 - accuracy: 0.8084
Epoch 5/5
391/391 [==============================] - 4s 10ms/step - loss: 0.4486 - accuracy: 0.828

In [16]:
# Train with half precision
time_half = 0.0
for _ in range(NUM_TRAINING_RUNS):
    model_half, training_time = build_and_train(train_dataset, encoder, 'half')
    time_half += training_time
accuracy_half = model_half.evaluate(test_dataset, verbose=2)[1]

Epoch 1/5
391/391 [==============================] - 55s 139ms/step - loss: 0.6659 - accuracy: 0.6795
Epoch 2/5
391/391 [==============================] - 43s 110ms/step - loss: 0.5616 - accuracy: 0.7579
Epoch 3/5
391/391 [==============================] - 40s 102ms/step - loss: 0.5504 - accuracy: 0.7755
Epoch 4/5
391/391 [==============================] - 40s 102ms/step - loss: 0.5119 - accuracy: 0.7929
Epoch 5/5
391/391 [==============================] - 38s 97ms/step - loss: 0.4980 - accuracy: 0.7986
Epoch 1/5
391/391 [==============================] - 50s 125ms/step - loss: 0.7468 - accuracy: 0.6613
Epoch 2/5
391/391 [==============================] - 44s 112ms/step - loss: 0.6273 - accuracy: 0.7326
Epoch 3/5
391/391 [==============================] - 41s 104ms/step - loss: 0.6010 - accuracy: 0.7509
Epoch 4/5
391/391 [==============================] - 37s 95ms/step - loss: 0.6253 - accuracy: 0.7626
Epoch 5/5
391/391 [==============================] - 40s 102ms/step - loss: 0.5401 -

In [17]:
# Train with mixed half precision
time_mixed = 0.0
for _ in range(NUM_TRAINING_RUNS):
    model_mixed, training_time = build_and_train_mixed(train_dataset, encoder)
    time_mixed += training_time
accuracy_mixed = model_mixed.evaluate(test_dataset, verbose=2)[1]

Epoch 1/5
391/391 [==============================] - 13s 29ms/step - loss: 0.6831 - accuracy: 0.5934
Epoch 2/5
391/391 [==============================] - 5s 13ms/step - loss: 0.6287 - accuracy: 0.7136
Epoch 3/5
391/391 [==============================] - 5s 12ms/step - loss: 0.5554 - accuracy: 0.7757
Epoch 4/5
391/391 [==============================] - 5s 11ms/step - loss: 0.4941 - accuracy: 0.8097
Epoch 5/5
391/391 [==============================] - 5s 11ms/step - loss: 0.4487 - accuracy: 0.8304
Epoch 1/5
391/391 [==============================] - 13s 30ms/step - loss: 0.6825 - accuracy: 0.5856
Epoch 2/5
391/391 [==============================] - 5s 12ms/step - loss: 0.6265 - accuracy: 0.7200
Epoch 3/5
391/391 [==============================] - 5s 13ms/step - loss: 0.5535 - accuracy: 0.7760
Epoch 4/5
391/391 [==============================] - 4s 11ms/step - loss: 0.4935 - accuracy: 0.8111
Epoch 5/5
391/391 [==============================] - 5s 13ms/step - loss: 0.4485 - accuracy: 0.829

In [18]:
print("---RESULTS---")
print("Average training time in double precision:", time_double / NUM_TRAINING_RUNS, "seconds")
print("Average training time in single precision:", time_single/ NUM_TRAINING_RUNS, "seconds")
print("Average training time in half precision:", time_half/ NUM_TRAINING_RUNS, "seconds")
print("Average training time in mixed half precision:", time_mixed/ NUM_TRAINING_RUNS, "seconds")
print("-------------")
print("Accuracy with double precision:", accuracy_double)
print("Accuracy with single precision:", accuracy_single)
print("Accuracy with half precision:", accuracy_half)
print("Accuracy with mixed half precision:", accuracy_mixed)

---RESULTS---
Average training time in double precision: 63.01487056414286 seconds
Average training time in single precision: 31.267759720484417 seconds
Average training time in half precision: 270.6589232285817 seconds
Average training time in mixed half precision: 42.05624405543009 seconds
-------------
Accuracy with double precision: 0.8341599702835083
Accuracy with single precision: 0.8341599702835083
Accuracy with half precision: 0.7747200131416321
Accuracy with mixed half precision: 0.8330399990081787
